In [31]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pathlib
import swifter
import matplotlib.pyplot as plt

from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

import geojson
import h3

DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../../..' )

In [136]:
# list of all metrices used in the analysis
metrics = [
    #'act_day', 
    #'act_stay',
    'act_sum',
    'centr_closes',
    'centr_eigen_w',
    'centr_eigen',
    'centr_infos',
    'day_sum',
    #'infl_dist',
    'infl_reg',
    'infl_sum',
    #'stay_avg',
    'stay_sum',
    'travel'
]

In [137]:
# merge all metrices with the list of destinations

df = pd.read_csv(DIR_ROOT + '/data/02_processed/destinations.csv', index_col='name')

for metric in metrics:
    df_metric = pd.read_csv(DIR_ROOT + '/data/05_metrics/' + metric + '.csv', index_col=0)
    df_metric.rename(columns={
       'p1': metric + '_p1', 
       'p2': metric + '_p2',
       'p3': metric + '_p3',
       'p4': metric + '_p4',
       'p5': metric + '_p5',
       'pall': metric + '',
    }, inplace=True)
    df_metric.drop(columns=['geometry'], inplace=True) 
    df_metric = (df_metric - df_metric.min()) / (df_metric.max() - df_metric.min())

    # merge to the table with all destinations
    df = pd.concat([df, df_metric], axis=1, join="inner")

In [157]:
df['median'] = df.apply(
    lambda x: x[metrics].median(),
    axis=1
)
df.sort_values('median', ascending=False)

,Unnamed: 0,id,no_alle,no_activity,no_reise,no_bi,no_gg,x,y,prazision,...,stay_sum_p4,stay_sum_p5,stay_sum,travel_p1,travel_p2,travel_p3,travel_p4,travel_p5,travel,median
Praha,420,421,399,171,175,44,9,50.09,14.41,1.0,...,1.0000,1.0,1.00,NaN,1.000000,1.000000,0.653875,0.427733,1.000000,1.000000
Wien,631,632,268,116,124,20,8,48.22,16.39,1.0,...,0.3750,NaN,0.58,0.371950,0.667121,0.748067,0.416228,0.000000,0.682007,0.717671
Brno,43,44,105,43,49,9,4,49.19,16.61,1.0,...,0.5625,1.0,0.38,0.000000,0.358439,0.341683,0.699586,1.000000,0.716256,0.608053
Krems an der Donau,228,229,72,30,30,5,7,48.41,15.62,1.0,...,0.0625,NaN,0.26,0.592467,0.617057,0.000000,0.327419,0.000000,0.487123,0.306374
Písek,395,396,28,13,13,1,1,49.31,14.15,1.0,...,0.0000,NaN,0.14,0.000000,0.165736,0.417519,0.072589,0.000000,0.264543,0.296142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lubljana,269,270,3,1,1,1,0,46.05,14.51,1.0,...,0.0000,NaN,0.00,NaN,NaN,NaN,0.075101,NaN,0.111752,0.002976
Kravaře u České Lípy,227,228,3,1,1,1,0,50.63,14.39,1.0,...,NaN,NaN,0.00,NaN,0.000000,0.011851,0.000000,0.000000,0.005825,0.002132
Most,312,313,6,1,1,2,2,50.50,13.64,1.0,...,NaN,NaN,0.00,NaN,0.000000,0.012717,0.000000,0.000000,0.006429,0.001392
Friesach,112,113,11,4,4,2,1,46.98,14.33,1.0,...,0.0000,NaN,0.00,NaN,NaN,0.000000,0.018628,NaN,0.026040,0.000766


In [139]:
# get order for each metric

df_matrices_order = pd.DataFrame(index=df.index)

for metric in metrics:
    df_copy = df.copy()
    df_copy.sort_values(metric, inplace=True, ascending=False)
    df_copy.reset_index(inplace=True)
    df_copy['rank'] = df_copy.index.values
    df_copy.set_index('index', inplace=True)
    df_matrices_order[metric] = df_copy[['rank']] + 1

df_matrices_order.loc['Praha']

act_sum          1
centr_closes     1
centr_eigen_w    1
centr_eigen      1
centr_infos      1
day_sum          1
infl_reg         1
infl_sum         1
stay_sum         1
travel           1
Name: Praha, dtype: int64

In [242]:
import altair as alt

max_rank = 5

# get list of important places
imp_places = []
for metric in metrics:
    imp_places_m = df_matrices_order[df_matrices_order[metric] <= max_rank].index
    imp_places.extend(imp_places_m)

imp_places = np.unique(np.array(imp_places))


imp_places = df_matrices_order.loc[imp_places]
imp_places['name'] = imp_places.index

alt.renderers.set_embed_options(
    padding={"left": 5, "right": 10, "bottom": 5, "top": 10}
)

rank_chart = alt.Chart(imp_places).transform_window(
    index='count()'
).transform_fold(
    metrics
).properties(
    width=500,
).encode(
    color='name:N',
    x=alt.X(
        'key:N',
        axis=alt.Axis(title='metrics')
    ),
    y=alt.Y(
        'value:Q', 
        scale=alt.Scale(zero=False, domain=[max_rank + 0.5,0.5]),
        axis=alt.Axis(values=[1,2,3,4,5], title='rank', tickMinStep=1, format='.0f'),
    ),
)

lines = rank_chart.mark_line(
    clip=True
)

circles = rank_chart.mark_circle(
    clip=True,
    size=100
)

lines + circles

alt.LayerChart(...)